# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
M. Fouesneau  ->  M. Fouesneau  |  ['M. Fouesneau']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


J. Liu  ->  J. Liu  |  ['J. Liu']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
Arxiv has 68 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2503.21863


extracting tarball to tmp_2503.21863...

 done.


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2503.21863/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

Found 74 bibliographic references in tmp_2503.22559/EC-UCD.bbl.
Retrieving document from  https://arxiv.org/e-print/2503.22642


extracting tarball to tmp_2503.22642...

 done.


L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']


Found 11 bibliographic references in tmp_2503.22642/sample631.bbl.
Found 98 bibliographic references in tmp_2503.22642/main.bbl.
Issues with the citations
syntax error in line 361: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.21863-b31b1b.svg)](https://arxiv.org/abs/2503.21863) | **The dawn of disks: unveiling the turbulent ionised gas kinematics of the galaxy population at $z\sim4-6$ with JWST/NIRCam grism spectroscopy**  |
|| A. L. Danhaive, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2025-03-31*|
|*Comments*| *25 pages, 13 figures, 3 tables, submitted to MNRAS*|
|**Abstract**|            Recent studies of gas kinematics at high redshift have reported disky systems which appear to challenge models of galaxy formation, but it is unclear whether they are representative of the underlying galaxy population. We present the first statistical sample of spatially resolved ionised gas kinematics at high redshift, comprised of $272$ H$\alpha$ emitters in GOODS-S and GOODS-N at redshifts $z\approx3.9-6.5$, observed with JWST/NIRCam slitless spectroscopy and imaging from JADES, FRESCO and CONGRESS. The sample probes two orders of magnitude in stellar mass ($\log (M_{\star}[\mathrm{M}_{\odot}])\approx8-10$) and star formation rate ($\text{SFR}\approx0.3-100\thinspace M_{\odot}/$yr), and is representative down to $\log(M_{\star}[\mathrm{M}_{\odot}])\approx 9$. Using a novel inference tool, $\texttt{geko}$, we model the grism data to measure morphological and kinematic properties of the ionised gas, as probed by H$\alpha$. Our results are consistent with a decrease of the rotational support $v/\sigma_0$\ and increase of the velocity dispersion $\sigma_0$ with redshift, with $\sigma_0\approx100$ km/s and $v/\sigma_0\approx1-2$ at $z\approx3.9-6.5$. We study the relations between $\sigma_0$, and $v/\sigma_0$, and different star formation tracers and find a large scatter and diversity, with the strongest correlations between $\sigma_0$ and SFR and SFR surface density. The fraction of rotationally supported systems ($v/\sigma_0>1$) slightly increases with cosmic time, from $(36\pm6)\%$ to $(41\pm6)\%$ from $z\sim 5.5$ to $z\sim 4.5$, for galaxies with masses $9<\log(M_{\star}[\mathrm{M}_{\odot}])<10$. Overall, disks do not dominate the turbulent high-redshift galaxy population in the mass range probed by this work. When placed in the context of studies up to cosmic noon, our results are consistent with a significant increase of disk-like systems with cosmic time.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.21936-b31b1b.svg)](https://arxiv.org/abs/2503.21936) | **Binarity at LOw Metallicity (BLOeM): Enhanced multiplicity of early B-type dwarfs and giants at $Z=0.2\,{\rm Z}_\odot$**  |
|| J. I. Villaseñor, et al. -- incl., <mark>M. Fouesneau</mark>, <mark>H.-W. Rix</mark> |
|*Appeared on*| *2025-03-31*|
|*Comments*| *17 pages, 19 figures (additional 10 pages and 5 figures in appendices). Accepted for publication in A&A*|
|**Abstract**|            Early B-type stars ($M_i=8-15$ M$_\odot$) are frequently in multiple systems, as evidenced by spectroscopic campaigns in the Milky Way (MW) and the Large Magellanic Cloud (LMC). Previous studies have shown no strong metallicity dependence in the close-binary (a>10 au) fraction or orbital-period distributions between the MW's solar metallicity (Z$_\odot$) and that of the LMC (Z=0.5 Z$_\odot$). However, similar analyses in more metal-poor environments are still scarce. We focus on 309 early B-type stars (luminosity classes III-V) from the Binarity at LOw Metallicity campaign in the Small Magellanic Cloud (SMC, Z=0.2 Z$_\odot$) using VLT/FLAMES multi-epoch spectroscopy. By applying binary detection criteria consistent with previous works, we identify 153 stars (91 SB1, 59 SB2, 3 SB3) exhibiting significant radial-velocity (RV) variations, resulting in an observed multiplicity fraction of $f^{obs}_{mult}=50\pm3\%$. Using Monte Carlo simulations to account for observational biases, we infer an intrinsic close-binary fraction of $f_{mult}=80\pm8\%$. A Markov chain Monte Carlo analysis of the peak-to-peak RV distribution ($\Delta{\rm RV}_{\rm max}$) confirms a high multiplicity fraction of $f_{mult}=78\pm5\%$. These findings suggest a possible anti-correlation between metallicity and the fraction of close B-type binaries, with the SMC multiplicity fraction significantly exceeding previous measurements in the LMC and MW. The enhanced fraction of close binaries at SMC's low metallicity may have broad implications for massive-star evolution in the early Universe. More frequent mass transfer and envelope stripping could boost the production of exotic transients, stripped supernovae, gravitational-wave progenitors, and sustained UV ionising flux, potentially affecting cosmic reionisation. Theoretical predictions of binary evolution under metal-poor conditions will provide a key test of our results.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.22497-b31b1b.svg)](https://arxiv.org/abs/2503.22497) | **Euclid Quick Data Release (Q1): A photometric search for ultracool dwarfs in the Euclid Deep Fields**  |
|| M. Žerjal, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-03-31*|
|*Comments*| *17 pages, 14 figures*|
|**Abstract**|            We present a catalogue of more than 5000 new ultracool dwarf (UCD) candidates in the three Euclid Deep Fields in the Q1 data release. They range from late M to late T dwarfs, and include 1200 L and T dwarfs. More than 100 of them have been spectroscopically confirmed, with seven of them being T dwarfs. Our UCD selection criteria are based only on colour ($I_\mathrm{E}-Y_\mathrm{E}>2.5$). The combined requirement for optical detection and stringent signal-to-noise ratio threshold ensure a high purity of the sample, but at the expense of completeness, especially for T dwarfs. The detections range from magnitudes 19 and 24 in the near-infrared bands, and extend down to 26 in the optical band. The average surface density of detected UCDs on the sky is approximately 100 objects per $\mathrm{deg}^2$, including 20 L and T dwarfs per $\mathrm{deg}^2$. This leads to an expectation of at least 1.4 million ultracool dwarfs in the final data release of the Euclid Wide Survey, including at least 300,000 L dwarfs, and more than 2,600 T dwarfs, using the strict selection criteria from this work. We provide empirical Euclid colours as a function of spectral type, and a probability that an object with a given colour has a certain spectral type.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.22559-b31b1b.svg)](https://arxiv.org/abs/2503.22559) | **Euclid Quick Data Release (Q1) Ultracool dwarfs in the Euclid Deep Field North**  |
|| A. Mohandasan, et al. -- incl., <mark>E. Bañados</mark>, <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-03-31*|
|*Comments*| *14 pages, 9 figures, 2 tables*|
|**Abstract**|            Ultracool dwarfs (UCDs) encompass the lowest mass stars and brown dwarfs, defining the stellar substellar boundary. They have significant potential for advancing the understanding of substellar physics; however, these objects are challenging to detect due to their low luminosity. The wide coverage and deep sensitivity of the Euclid survey will increase the number of confirmed and well characterised UCDs by several orders of magnitude. In this study, we take advantage of the Euclid Quick Data Release (Q1) and in particular we look in detail at the known and new UCDs in the Euclid Deep Field North (22.9 deg2 down to JE = 24.5 mag), to understand the advantages of using the slitless Euclid spectroscopy. We compile a comparison sample of known UCDs and use their spectra to demonstrate the capability of Euclid to derive spectral types using a template matching method. This method is then applied to the spectra of the newly identified candidates. We confirm that 33 of these candidates are new UCDs, with spectral types ranging from M7 to T1 and JE = 17 to 21 mag. We look at their locus in colour colour diagrams and compare them with the expected colours of QSOs. A machine readable catalogue is provided for further study, containing both the comparison sample and the newly identified UCDs, along with their spectral classifications where the Q1 spectra quality allows for confident determination         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.22642-b31b1b.svg)](https://arxiv.org/abs/2503.22642) | **The radiative effects of photochemical hazes on the atmospheric circulation and phase curves of sub-Neptunes**  |
|| M. E. Steinrueck, et al. -- incl., <mark>L. Kreidberg</mark> |
|*Appeared on*| *2025-03-31*|
|*Comments*| *resubmitted to ApJ after addressing referee report*|
|**Abstract**|            Measuring the atmospheric composition of hazy sub-Neptunes like GJ~1214b through transmission spectroscopy is difficult because of the degeneracy between mean molecular weight and haziness. It has been proposed that phase curve observations can break this degeneracy because of the relationship between mean molecular weight (MMW) and phase curve amplitude. However, photochemical hazes can strongly affect phase curve amplitudes as well. We present a large set of GCM simulations of the sub-Neptune GJ~1214b that include photochemical hazes with varying atmospheric composition, haze opacity and haze optical properties. In our simulations, photochemical hazes cause temperature changes of up to 200~K, producing thermal inversions and cooling deeper regions. This results in increased phase curve amplitudes and adds a considerable scatter to the phase curve amplitude--metallicity relationship. However, we find that if the haze production rate is high enough to significantly alter the phase curve, the secondary eclipse spectrum will exhibit either emission features or strongly muted absorption features. Thus, the combination of a white-light phase curve and a secondary eclipse spectrum can successfully distinguish between a hazy, lower MMW and a clear, high MMW scenario.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.22331-b31b1b.svg)](https://arxiv.org/abs/2503.22331) | **AT2021uey: A planetary microlensing event outside the Galactic bulge**  |
|| M. Ban, et al. |
|*Appeared on*| *2025-03-31*|
|*Comments*| *10 pages, 5 figures, 6 tables. Accepted by A&A*|
|**Abstract**|            We report the analysis of a planetary microlensing event AT2021uey. The event was observed outside the Galactic bulge and was alerted by both space- (Gaia) and ground-based (ZTF and ASAS-SN) surveys. From the observed data, we find that the lens system is located at a distance of 1 kpc and comprises an M-dwarf host star of about half a solar mass, orbited by a Jupiter-like planet beyond the snowline. The source star could be a metal-poor giant located in the halo according to the spectral analyses and modelling. Hence, AT2021uey is a unique example of the binary-lens event outside the bulge that is offered by a disc-halo lens-source combination.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.22127-b31b1b.svg)](https://arxiv.org/abs/2503.22127) | **Asteroseismology of Long-Period Variables with OGLE-IV data: Using Global Seismic Parameters as Luminosity Indicators**  |
|| Q. Xiong, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-03-31*|
|*Comments*| *18 pages, 9 figures in the main text, 2 figures in the appendix*|
|**Abstract**|            Long-period variables (LPVs) are high-luminosity red giants or supergiants with pulsation periods ranging from days to years. Many LPVs in the Large Magellanic Cloud (LMC) and Galactic Bulge (BLG) have been continuously observed over a time span of 26 years by the Optical Gravitational Lensing Experiment (OGLE) survey. Using OGLE-IV data, we applied Gaussian Processes with kernels tailored for solar-like oscillations to extract two global asteroseismic parameters: the frequency of maximum power (numax) and the large frequency separation (Dnu), for LPVs with primary mode periods (P1) between 10 and 100 days in the LMC and BLG. We found that the numax-Dnu relation for LPVs in this work aligns with that of lower-luminosity Kepler red giants, confirming that the pulsations of these LPVs are likely solar-like. We found that numax and Dnu can serve as luminosity indicators. Compared to P1, numax and Dnu exhibit significantly tighter correlations with the absolute magnitude in the 2MASS K_s band (M_{K}), with corresponding scatter of 0.27 mag and 0.21 mag, respectively. Using the calibrated numax-mk and Dnu-mk relations for LPVs in the LMC, we determined the M_{K} values for individual stars in the BLG. By accounting for extinction, we further calculated the distances to 4,948 BLG stars. The peak of the resulting distance distribution corresponds to an estimated distance to the Galactic center of approximately 9.1 kpc, which appears to be overestimated, suggesting that the seismic luminosity relation calibrated from the LMC may not be directly applicable to BLG stars.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.22466-b31b1b.svg)](https://arxiv.org/abs/2503.22466) | **Properties of Interstellar Medium in the S0 Galaxy NGC 1222: Evidence for Shocked-enhanced Line Emission**  |
|| <mark>J. Liu</mark>, Y. Zhao, K.-X. Lu, J.-M. Bai |
|*Appeared on*| *2025-03-31*|
|*Comments*| *Accepted for publication in ApJ*|
|**Abstract**|            In this paper we present a comprehensive study on the properties of the interstellar medium in NGC~1222, a star-forming early-type merging galaxy that forms a triple system, using optical and far-infrared (FIR) spectroscopic, and multiband photometric data. The fit to the spectral energy distribution reveals a high dust content in the galaxy, with a dust-to-stellar mass ratio of $M_\mathrm{dust}/M_\star\sim3.3\times10^{-3}$ that is 40$-$90 larger than the mean value of local S0 galaxies. By comparing the observed optical emission line ratios to shock models, we suggest that a merger-induced shock, which is further supported by the higher-than-average \OI- and \CII-to-PAH ratios, plays a role in heating the gas in NGC~1222. We also show evidence for gas inflow by analysing the kinematic properties of NGC~1222.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2503.21863/./Figures/sigma0_z_comparison_fitlegend.png', 'tmp_2503.21863/./Figures/vsigma_sigma_mstar_sfr_masscolor.png', 'tmp_2503.21863/./Figures/vsigma_sigma_mstar_sfr_sigsfr_litcomp.png']
copying  tmp_2503.21863/./Figures/sigma0_z_comparison_fitlegend.png to _build/html/
copying  tmp_2503.21863/./Figures/vsigma_sigma_mstar_sfr_masscolor.png to _build/html/
copying  tmp_2503.21863/./Figures/vsigma_sigma_mstar_sfr_sigsfr_litcomp.png to _build/html/
exported in  _build/html/2503.21863.md
    + _build/html/tmp_2503.21863/./Figures/sigma0_z_comparison_fitlegend.png
    + _build/html/tmp_2503.21863/./Figures/vsigma_sigma_mstar_sfr_masscolor.png
    + _build/html/tmp_2503.21863/./Figures/vsigma_sigma_mstar_sfr_sigsfr_litcomp.png
found figures ['tmp_2503.21936/./img/bloem_dRV_Pcrit.png', 'tmp_2503.21936/./img/5_062_4388_fits_SB2_.png', 'tmp_2503.21936/./img/spectral_type_hist_byField_magma.png']
copying  tmp_2503.21936/./img/bloem_dRV_Pcrit.png to _build/html/
copying  

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\OIII}{[\text{OIII}]}$
$\newcommand{\Ha}{H\alpha}$
$\newcommand{\disp}{\sigma_0}$
$\newcommand{\reff}{r_{e}}$
$\newcommand{\rotsupp}{\text{v}/\sigma_0}$
$\newcommand{\logMstar}{\log\thinspace M_{\star}}$
$\newcommand{\Mstar}{M_{\star}}$
$\newcommand{\logMdyn}{\log\thinspace M_{\text{dyn}}}$
$\newcommand{\Mdyn}{M_{\text{dyn}}}$
$\newcommand{\PAmorph}{\text{PA}_{\text{morph}}}$
$\newcommand{\PAkin}{\text{PA}_{\text{kin}}}$
$\newcommand{\vre}{\text{v}_{\text{re}}}$
$\newcommand{\vcirc}{\text{v}_{\text{circ}}}$
$\newcommand{\geko}{\textsc{geko}}$
$\newcommand$
$\newcommand{\labelitemi}{\labelitemii}$
$\newcommand{\arraystretch}{2}$
$\newcommand{\arraystretch}{2}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

#  The dawn of disks: unveiling the turbulent ionised gas kinematics of the galaxy population at $z\sim4-6$ with JWST/NIRCam grism spectroscopy

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.21863-b31b1b.svg)](https://arxiv.org/abs/2503.21863)<mark>Appeared on: 2025-03-31</mark> -  _25 pages, 13 figures, 3 tables, submitted to MNRAS_

</div>
<div id="authors">

A. L. Danhaive, et al. -- incl., <mark>A. d. Graaff</mark>

</div>
<div id="abstract">

**Abstract:** Recent studies of gas kinematics at high redshift have reported disky systems which appear to challenge models of galaxy formation, but it is unclear whether they are representative of the underlying galaxy population. We present the first statistical sample of spatially resolved ionised gas kinematics at high redshift, comprised of $272$ H $\alpha$ emitters in GOODS-S and GOODS-N at redshifts $z\approx3.9-6.5$ , observed with JWST/NIRCam slitless spectroscopy and imaging from JADES, FRESCO and CONGRESS. The sample probes two orders of magnitude in stellar mass ( $\log (M_{\star}[\mathrm{M}_{\odot}])\approx8-10$ ) and star formation rate ( $\text{SFR}\approx0.3-100\thinspace M_{\odot}/$ yr), and is representative down to $\log(M_{\star}[\mathrm{M}_{\odot}])\approx 9$ . Using a novel inference tool, geko , we model the grism data to measure morphological and kinematic properties of the ionised gas, as probed by H $\alpha$ . Our results are consistent with a decrease of the rotational support $v/\sigma_0$ and increase of the velocity dispersion $\sigma_0$ with redshift, with $\sigma_0\approx100$ km/s and $v/\sigma_0\approx1-2$ at $z\approx3.9-6.5$ . We study the relations between $\sigma_0$ , and $v/\sigma_0$ , and different star formation tracers and find a large scatter and diversity, with the strongest correlations between $\sigma_0$ and SFR and SFR surface density. The fraction of rotationally supported systems ( $v/\sigma_0>1$ ) slightly increases with cosmic time, from $(36\pm6)\%$ to $(41\pm6)\%$ from $z\sim 5.5$ to $z\sim 4.5$ , for galaxies with masses $9<\log(M_{\star}[\mathrm{M}_{\odot}])<10$ . Overall, disks do not dominate the turbulent high-redshift galaxy population in the mass range probed by this work. When placed in the context of studies up to cosmic noon, our results are consistent with a significant increase of disk-like systems with cosmic time.

</div>

<div id="div_fig1">

<img src="tmp_2503.21863/./Figures/sigma0_z_comparison_fitlegend.png" alt="Fig14" width="100%"/>

**Figure 14. -** Intrinsic $\Ha$ velocity dispersion $\disp$ as a function of redshift for our gold (red outlined squares) and silver (light gray circles) in the context of studies of warm ionised gas (i.e. $\Ha$ and [OIII]; orange outlines) and cold gas (i.e. HI, CO, and [CII]; blue outlines) kinematics across redshifts. The medians for the gold and extended samples are plotted in red squares and purple diamonds, respectively. The errorbars on the medians represent the $16^{\rm th}$ and $84^{\rm th}$ quantiles along the y-axis, and the extent of the bin along the x-axis. Our results are consistent with a decrease of $\disp$ with cosmic time when compared to  surveys from cosmic noon (KMOS3D \protect ([ and Wisnioski 2015](https://ui.adsabs.harvard.edu/abs/2015ApJ...799..209W)) , KROSS \protect ([ and Johnson 2018](https://ui.adsabs.harvard.edu/abs/2018MNRAS.474.5076J)) , MOSDEF  ([ and Price 2016](https://ui.adsabs.harvard.edu/abs/2016ApJ...819...80P), [ and Price 2020](https://ui.adsabs.harvard.edu/abs/2020ApJ...894...91P)) , KDS \protect ([ and Turner 2017](https://ui.adsabs.harvard.edu/abs/2017MNRAS.471.1280T), [Turner, et. al 2017](https://ui.adsabs.harvard.edu/abs/2017arXiv171103604T)) , SIGMA \protect ([ and Simons 2016](https://ui.adsabs.harvard.edu/abs/2016ApJ...830...14S)) , SINS/zC-SINF \protect (Forster-Schreiber:2006wk,  Forster-Schreiber:2018aa) , PHIBSS \protect ([ and Tacconi 2013](https://ui.adsabs.harvard.edu/abs/2013ApJ...768...74T)) , and points from \protect ([ and Swinbank 2011](https://ui.adsabs.harvard.edu/abs/2011ApJ...742...11S), [ and Genzel 2017](https://ui.adsabs.harvard.edu/abs/2017Natur.543..397G)) ) and the local Universe (GHASP \protect ([Epinat, et. al 2010](https://ui.adsabs.harvard.edu/abs/2010MNRAS.401.2113E)) , DYNAMO \protect ([ and Green 2014](https://ui.adsabs.harvard.edu/abs/2014MNRAS.437.1070G)) , EDGE-CALIFA \protect ([ and Bolatto 2017](https://ui.adsabs.harvard.edu/abs/2017ApJ...846..159B)) , HERACLES \protect ([ and Leroy 2009](https://ui.adsabs.harvard.edu/abs/2009AJ....137.4670L)) , THINGS \protect ([Walter, et. al 2008](https://ui.adsabs.harvard.edu/abs/2008AJ....136.2563W)) , and \protect ([Dib, Bell and Burkert 2006](https://ui.adsabs.harvard.edu/abs/2006ApJ...638..797D)) ; blue shaded region). Our best-fit relations (Eq. \ref{eq:powerlaw}), with (purple) and without (red) including upper limits, agree with the fit from \protect[ and Übler (2019)](https://ui.adsabs.harvard.edu/abs/2019ApJ...880...48U)(light red - long dashes), which is defined out to $z\sim4$. Similarly, we also include the exponential fit for cold gas measurements from \protect[ and Rizzo (2024)](https://ui.adsabs.harvard.edu/abs/2024A&A...689A.273R)(dark blue - long dashes) defined out to $z\sim5$, which highlight a similar evolution of $\disp$ tracing the cold gas, albeit with a factor 2-3 lower normalization. We compare our measurements with works at similar redshifts \protect ([Parlanti, et. al 2023](https://ui.adsabs.harvard.edu/abs/2023A&A...673A.153P), [ and Arribas 2024](https://ui.adsabs.harvard.edu/abs/2024A&A...688A.146A), de-Graaff:2024ab) , as well as detection of dynamically cold systems at high-redshift \protect[Neeleman, et. al (2020)](https://ui.adsabs.harvard.edu/abs/2020Natur.581..269N), [Rizzo, et. al (2020)](https://ui.adsabs.harvard.edu/abs/2020Natur.584..201R), [Lelli, et. al (2021)](https://ui.adsabs.harvard.edu/abs/2021Sci...371..713L), [ and Pope (2023)](https://ui.adsabs.harvard.edu/abs/2023ApJ...951L..46P), [ and Rowland (2024)](https://ui.adsabs.harvard.edu/abs/2024MNRAS.535.2068R). (*fig:sigma0-z-comp*)

</div>
<div id="div_fig2">

<img src="tmp_2503.21863/./Figures/vsigma_sigma_mstar_sfr_masscolor.png" alt="Fig16" width="100%"/>

**Figure 16. -** Dependence of the intrinsic velocity dispersion $\disp$(top panels) and rotational support $\rotsupp$(bottom panels) on $M_{\star}$, $\text{SFR}_{10}$, $\text{sSFR}_{10}$, and SFR surface density $\Sigma_{\text{SFR}_{10}}$ for our gold (squares) and silver sample (gray contours). We find a significant correlation between $\disp$ and $\text{SFR}_{10}$, which is highlighted by the running medians, for the gold sample (red squares) and for the extended sample (purple diamonds), and the Spearman rank coefficient $\rho$ and p-value $p$ for the resolved sample. The errorbars on the medians represent the $16^{\rm th}$ and $84^{\rm th}$ quantiles along the y-axis, and the extent of the bin along the x-axis. Both $\disp$ and $\rotsupp$ show a (weaker) correlation with $\Sigma_{\text{SFR}_{10}}$, highlighting the role of surface density in efficiently driving turbulence in the gas. The trends with $\text{SFR}_{10}$ and $\Sigma_{\text{SFR}_{10}}$ are stronger for $\disp$ than $\rotsupp$, which could be caused by the fact that we probing many dispersion-supported systems ($\rotsupp<1$) and systems in the transitional regions (green shaded region) between dispersion and rotation support, where stable rotation has not yet been established. (*fig:vsigma_sigma_mstar_sfr*)

</div>
<div id="div_fig3">

<img src="tmp_2503.21863/./Figures/vsigma_sigma_mstar_sfr_sigsfr_litcomp.png" alt="Fig17" width="100%"/>

**Figure 17. -** Dependence of $\disp$(top panels) and $\rotsupp$(bottom panels) on stellar mass and SFR for our sample (red squares and purple diamonds) in the context of other high-redshift works \citep[in red;][]{Parlanti:2023ab, de-Graaff:2024ab}, and works at cosmic noon (in green; see Fig. \ref{fig:sigma0-z-comp}) and the local Universe (in blue; MANGA  ([ and Yu 2019](https://ui.adsabs.harvard.edu/abs/2019MNRAS.486.4463Y)) , SAMI  ([ and Allen 2015](https://ui.adsabs.harvard.edu/abs/2015MNRAS.446.1567A), [ and Johnson 2018](https://ui.adsabs.harvard.edu/abs/2018MNRAS.474.5076J), [ and Green 2018](https://ui.adsabs.harvard.edu/abs/2018MNRAS.475..716G)) , and MUSE  ([ and Swinbank 2017](https://ui.adsabs.harvard.edu/abs/2017MNRAS.467.3140S)) ). The errorbars on our plotted medians represent the $16^{\rm th}$ and $16^{\rm th}$ quantiles along the y-axis, and the extent of the bin along the x-axis. The $\disp$-SFR relation holds across redshifts, for both ionised ($\Ha$; orange outline) and cold ([CII]; blue outline) gas, although we find higher values of $\disp$ and a steeper slope. Some works at lower redshift find a similar weak increasing trend for the $\disp-M_{\star}$ relation as our sample  ([ and Wisnioski 2015](https://ui.adsabs.harvard.edu/abs/2015ApJ...799..209W), Forster-Schreiber:2018aa, [ and Yu 2019](https://ui.adsabs.harvard.edu/abs/2019MNRAS.486.4463Y), [ and Price 2020](https://ui.adsabs.harvard.edu/abs/2020ApJ...894...91P)) , while [ and Simons (2017)](https://ui.adsabs.harvard.edu/abs/2017ApJ...843...46S) finds no significant trend. The trends for $\rotsupp$ are less clear, but our results could indicate a flattening in the $\rotsupp-M_{\star}$ relation at low masses $\log M_{\star} [\mathrm{M}_{\odot}] \lesssim10$, which is consistent with other works in this mass range and which is predicted by the marginal disk model (purple shaded regions). (*fig:corr-comp*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.21863"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\jv}[1]{\textcolor{magenta}{JV: #1}}$
$\newcommand{\bloem}{BLOeM\xspace}$
$\newcommand{\kms}{ km s^{-1}\xspace}$
$\newcommand{\cms}{ cm s^{-2}\xspace}$
$\newcommand{\Msun}{ {\rm M}_\odot\xspace}$
$\newcommand{\Rsun}{ {\rm R}_\odot\xspace}$
$\newcommand{\Lsun}{ {\rm L}_\odot\xspace}$
$\newcommand{\Zsun}{ {\rm Z}_\odot\xspace}$
$\newcommand{\Teff}{T_{\rm eff}\xspace}$
$\newcommand{\logg}{\log g\xspace}$
$\newcommand{\drv}{\DeltaRV\xspace}$
$\newcommand{\drvmax}{\Delta{\rm RV}_{\rm max}\xspace}$
$\newcommand{\rvvar}{RV var\xspace}$
$\newcommand{\pdet}{p_{\rm det}\xspace}$
$\newcommand$</div>



<div id="title">

# Binarity at LOw Metallicity (BLOeM):$\thanks{Based on observations collected at the European Southern Observatory under ESO program ID 112.25W2.}$: Enhanced multiplicity of early B-type dwarfs and giants at $Z=0.2$$\Zsun$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.21936-b31b1b.svg)](https://arxiv.org/abs/2503.21936)<mark>Appeared on: 2025-03-31</mark> -  _17 pages, 19 figures (additional 10 pages and 5 figures in appendices). Accepted for publication in A&A_

</div>
<div id="authors">

J. I. Villaseñor, et al. -- incl., <mark>M. Fouesneau</mark>, <mark>H.-W. Rix</mark>

</div>
<div id="abstract">

**Abstract:** Early B-type stars with initial masses between 8 and 15 $\Msun$ are frequently found in multiple systems, as evidenced by multi-epoch spectroscopic campaigns in the Milky Way and the Large Magellanic Cloud (LMC). Previous studies have shown no strong metallicity dependence in the close-binary ( $a<10$ au) fraction or orbital-period distributions between the Milky Way's solar metallicity ( $\Zsun$ ) and that of the LMC ( $Z=0.5$ $\Zsun$ ). However, similar analyses for a large sample of massive stars in more metal-poor environments are still scarce.We focus on 309 early B-type stars (luminosity classes III–V) from the Binarity at LOw Metallicity ( $\bloem$ ) campaign, which targeted nearly 1000 massive stars in the Small Magellanic Cloud (SMC, $Z=0.2$ $\Zsun$ ) using VLT/FLAMES multi-epoch spectroscopy.By applying binary detection criteria consistent with previous works on Galactic and LMC samples, we identify 153 stars (91 SB1, 59 SB2, 3 SB3) exhibiting significant radial-velocity (RV) variations, resulting in an observed multiplicity fraction of $\fobs{50}{3}$ . Using Monte Carlo simulations to account for observational biases, we infer an intrinsic close-binary fraction of $\fbin{80}{8}$ . This fraction reduces to $\sim55$ \% when increasing our RV threshold from 20 to 80 $\kms$ , however, an independent Markov chain Monte Carlo analysis of the peak-to-peak RV distribution ( $\drvmax$ ) confirms a high multiplicity fraction of $\fbin{78}{5}$ . These findings suggest a possible anti-correlation between metallicity and the fraction of close B-type binaries, with the SMC multiplicity fraction significantly exceeding previous measurements in the LMC and the Galaxy.The enhanced fraction of close binaries at SMC's low metallicity may have broad implications for massive-star evolution in the early Universe. More frequent mass transfer and envelope stripping could boost the production of exotic transients, stripped supernovae, gravitational-wave progenitors, and sustained UV ionising flux, potentially affecting cosmic reionisation. Theoretical predictions of binary evolution under metal-poor conditions will provide a key test of our results.

</div>

<div id="div_fig1">

<img src="tmp_2503.21936/./img/bloem_dRV_Pcrit.png" alt="Fig4" width="100%"/>

**Figure 4. -** Top panel: Contour plot showing the critical period ($P_{\rm crit}$) as a function of primary mass ($M_1$) and surface gravity ($\log(g)$) for systems with $i=60^\circ$ and $q=0.5$. White dashed contour lines highlight selected $P_{\rm crit}$ values (1, 2, 5, and 10 days).
    Bottom Panel: As the top panel, this time displaying the corresponding change in radial velocity ($\Delta \mathrm{RV}_{\rm max}$) computed at $P_{\rm crit}$. White dashed curves mark the contours where $\Delta \mathrm{RV}_{\rm max} = 20$\kms at different inclinations and $q=0.1$. (*fig:Pcrit*)

</div>
<div id="div_fig2">

<img src="tmp_2503.21936/./img/5_062_4388_fits_SB2_.png" alt="Fig21" width="100%"/>

**Figure 21. -** Gaussian fitting of the spectral lines \spline{He}{i}{4388} for the SB3 system $\bloem$ 5-062. Red dashed line shows the mean SMC velocity of 172$\kms$\citep{evans+howarth08}, and orange dashed lines show differences of $\pm200$\kms with respect to the mean velocity. Epoch number increases from left to right and from top to bottom. (*fig:5_062_SB3*)

</div>
<div id="div_fig3">

<img src="tmp_2503.21936/./img/spectral_type_hist_byField_magma.png" alt="Fig1" width="100%"/>

**Figure 1. -** Spectral types of the B-type III/V sample, colour coded by field. Number of stars per field is shown in parentheses. (*fig:SpTs*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.21936"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]$
$\newcommand{\tmax}{{\vartheta_{\rm max}}}$
$\newcommand{\refer}[1]{,\ \cite{#1}; [{\tt #1}]}$
$\newcommand{\itemm}{$
$\medskip\noindent}$
$\newcommand{\pd}{\phantom{1}}$</div>



<div id="title">

# $\Euclid$\/: Title of your paper$\thanks{This paper is published on     behalf of the Euclid Consortium}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.22497-b31b1b.svg)](https://arxiv.org/abs/2503.22497)<mark>Appeared on: 2025-03-31</mark> -  _17 pages, 14 figures_

</div>
<div id="authors">

M. Žerjal, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** ${\color{blue} \bf NEW VERSION: We have a modified style file,     aaEC.cls, used here; with that, the bug reported in Sect. 8 no     longer occurs. In order to demonstrate this, the long author list     of \cite{EROData} was copied here.}$ \ This is a template file for the preparation of Euclid Consortium (EC) papersfor submission to A \& A. The list of lead authors and affiliations shouldbe prepared by the authors only on first submission to the ECEditorial Board (ECEB). Once thepaper is circulated within the full EC, and eligible authors can signup, then you should instead use the author and affiliation listgenerated for your paper on the Publication Portal; any editing ofthese lists byhand is error prone and must not be done.With the macros \textbackslash{\tt titlerunning} and \textbackslash{\tt authorrunning} ,shortened versions of the author list and the title should be given, suchthat they fit into the headline of each odd-numbered page.As a default, it is recommended to use the "traditional" abstract, asgiven in this example here. Nevertheless, the structure of theabstract should follow the journal's guidelines, i.e., context, aims,methods, and results, just without explicitly writing the headings.In the Appendix, we provide a list of EC publications, which will becontinuously updated. Furthermore, we have prepared the file ${\tt  Euclid.bib}$ , which contains most of the EC papers and can be usedfor preparing your reference list.

</div>

<div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.22497"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\com}[2]{ {\textcolor{orange}{ #1 :}}{\textcolor{blue}{ #2}}}$
$\newcommand{\NNEW}{202\xspace}$
$\newcommand{\NNEWSPECTRA}{33\xspace}$
$\newcommand{\NKNWSPECTRA}{109\xspace}$
$\newcommand{\NSPECTRA}{142\xspace}$
$\newcommand{\NNEWINTSPECTRA}{46\xspace}$
$\newcommand{\NDES}{326\xspace}$
$\newcommand{\NKNW}{12\xspace}$
$\newcommand{\EDFN}{EDF-N\xspace}$
$\newcommand{\EDFF}{EDF-F\xspace}$
$\newcommand{\EDFS}{EDF-S\xspace}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# $\Euclid$ Quick Data Release (Q1) --Ultracool dwarfs in the Euclid Deep Field North$\thanks{This paper is published on behalf of the Euclid Consortium}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.22559-b31b1b.svg)](https://arxiv.org/abs/2503.22559)<mark>Appeared on: 2025-03-31</mark> -  _14 pages, 9 figures, 2 tables_

</div>
<div id="authors">

A. Mohandasan, et al. -- incl., <mark>E. Bañados</mark>, <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** Ultracool dwarfs (UCDs) encompass the lowest-mass stars and brown dwarfs, defining the stellar-substellar boundary. They have significant potential for advancing the understanding of substellar physics; however, these objects are challenging to detect due to their low luminosity. The wide coverage and deep sensitivity of the $\Euclid$ survey will increase the number of confirmed and well-characterised UCDs by several orders of magnitude.In this study, we take advantage of the $\Euclid$ Quick Data Release (Q1) and in particular we look in detail at the known and new UCDs in the $\Euclid$ Deep Field North ( $22.9\deg^2$ down to $\JE$ $\approx 24.5$ mag), to understand the advantages of using the slitless $\Euclid$ spectroscopy.We compile a comparison sample of known UCDs and use their spectra to demonstrate the capability of $\Euclid$ to derive spectral types using a template-matching method. This method is then applied to the spectra of the newly identified candidates.We confirm that $\NNEWSPECTRA$ of these candidates are new UCDs, with spectral types ranging from M7 to T1 and $\JE=17$ --21 mag.We look at their locus in colour-colour diagrams and compare them with the expected colours of QSOs.A machine-readable catalogue is provided for further study, containing both the comparison sample and the newly identified UCDs, along with their spectral classifications where the Q1 spectra quality allows for confident determination.

</div>

<div id="div_fig1">

<img src="tmp_2503.22559/./Images/gridplot_lit5.png" alt="Fig1" width="100%"/>

**Figure 1. -** Q1 spectra of four UCDs with published spectral classifications. The solid green line represents the template spectrum corresponding to the published spectral type. Grey points denote all measured data, while blue points indicate those retained after quality filtering. (*fig:maskeffect*)

</div>
<div id="div_fig2">

<img src="tmp_2503.22559/./Images/J18196557_2.png" alt="Fig3" width="100%"/>

**Figure 3. -** Peculiar spectrum of J1813$+$6557, along with spectra of very low-gravity and a normal L5 from JWST and a very low-gravity L7 observed from the ground, for comparison (refer to Sect. \ref{spectral_new}). Spectra are normalised at 1.7 \micron . (*fig:speactra_new*)

</div>
<div id="div_fig3">

<img src="tmp_2503.22559/./Images/check_IHvsIJ_QSOT.png" alt="Fig7.1" width="25%"/><img src="tmp_2503.22559/./Images/check_YJvsJH_QSOT.png" alt="Fig7.2" width="25%"/><img src="tmp_2503.22559/./Images/check_IHvsYJ_QSOT.png" alt="Fig7.3" width="25%"/><img src="tmp_2503.22559/./Images/check_JHvsIY_QSOT.png" alt="Fig7.4" width="25%"/>

**Figure 7. -** 142 UCDs analysed in this study, of which 33 are newly identified with \Euclid, displayed in various colour-colour diagrams, with only those having good quality spectrum and confident spectral typing included. Dots represent previously known UCDs with good-quality Q1 spectra. Diamonds represent new UCDs studied in this work with reliable spectral typing. They are colour-coded according to the spectral type assigned in this paper, which is displayed in the colour scale. Grey points are all Q1 objects with \texttt{point}\_\texttt{like}\_\texttt{probab}$\geq 0.8$ with a shade that represents the number density as indicated in the scale.
The dashed lines of different colours denote the expected position for typical QSOs with redshift from $z=5$ to $z=9$; White for default QSO position. Light and dark blue denote extremely weak and extremely strong lined QSO. Light and dark red denote *E(B$-$V)*=0.1 and 0.2, in QSO rest frame. Light and dark green denote extremely weak and strong lined QSO with *E(B$-$V)*=0.1.
 (*fig:Q1x:YJvsJH*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.22559"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\MalskyEtAlP}{\citep{MalskyEtAl2025}}$
$\newcommand{\MalskyEtAlT}{\citet{MalskyEtAl2025} }$</div>



<div id="title">

# The radiative effects of photochemical hazes on the atmospheric circulation and phase curves of sub-Neptunes

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.22642-b31b1b.svg)](https://arxiv.org/abs/2503.22642)<mark>Appeared on: 2025-03-31</mark> -  _resubmitted to ApJ after addressing referee report_

</div>
<div id="authors">

M. E. Steinrueck, et al. -- incl., <mark>L. Kreidberg</mark>

</div>
<div id="abstract">

**Abstract:** Measuring the atmospheric composition of hazy sub-Neptunes like GJ 1214b through transmission spectroscopy is difficult because of the degeneracy between mean molecular weight and haziness. It has been proposed that phase curve observations can break this degeneracy because of the relationship between mean molecular weight (MMW) and phase curve amplitude. However, photochemical hazes can strongly affect phase curve amplitudes as well. We present a large set of GCM simulations of the sub-Neptune GJ 1214b that include photochemical hazes with varying atmospheric composition, haze opacity and haze optical properties. In our simulations, photochemical hazes cause temperature changes of up to 200 K, producing thermal inversions and cooling deeper regions. This results in increased phase curve amplitudes and adds a considerable scatter to the phase curve amplitude--metallicity relationship. However, we find that if the haze production rate is high enough to significantly alter the phase curve, the secondary eclipse spectrum will exhibit either emission features or strongly muted absorption features. Thus, the combination of a white-light phase curve and a secondary eclipse spectrum can successfully distinguish between a hazy, lower MMW and a clear, high MMW scenario.

</div>

<div id="div_fig1">

<img src="tmp_2503.22642/./uzonalav_clear.png" alt="Fig7" width="100%"/>

**Figure 7. -** Zonal-mean zonal velocity for the clear-atmosphere simulations. The contours highlight where the zonal-mean zonal velocity reaches 50\% and 75\% of the peak velocity within the simulation. (*fig:uzonalav_clear*)

</div>
<div id="div_fig2">

<img src="tmp_2503.22642/./hazeproperties_vs_pressure.png" alt="Fig5" width="100%"/>

**Figure 5. -** Summary of the haze properties assumed in the simulations. The left panel shows number density and mean radius of the haze distribution that was used as input in the GCM simulations. The middle panel shows the differential optical depth for soot (solid) and tholin (dashed) hazes at 1 $\mu$m (representative for the incoming starlight) and 8 $\mu$m (representative for the outgoing thermal radiation) for the nominal haze opacity case. The pressure levels where the optical depth reaches 0.01 (circle), 0.1 (star), and 1 (square) are highlighted. For soot at 1 $\mu$m, the optical depth reaches 0.01 at a pressure of $6\cdot10^{-7}$ bar, outside the pressure range shown.  The right panel shows the wavelength-averaged single scattering albedo weighted by the stellar flux. For the "maximally reflective" case, the optical depth is identical to the soot case, but the single scattering albedo is fixed to 0.9999. (*fig:hazeproperties_vs_pressure*)

</div>
<div id="div_fig3">

<img src="tmp_2503.22642/./spectralphasecurve_tholin.png" alt="Fig15" width="100%"/>

**Figure 15. -** Spectral phase curves for a selection of models with tholin hazes that provide a relatively good match to the observed white-light phase curve. The models substantially overpredict the emitted flux for a large fraction of the phase curve in spectral regions that show emission features in the secondary eclipse spectrum (6-8.5 $\mu$m). The observed spectral phasecurves are plotted in the background in light gray with filled circles and solid lines representing the \citet{KemptonEtAl2023} reduction and empty circles and dashed lines the $\MalskyEtAlT$ reduction. We note that due to the relatively low resolution of our post-processing, the spectral bins of the models do not line up exactly with the spectral bins of the observations (see Appendix for more detail). (*fig:spectralphasecurve_tholin*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.22642"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

417  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

21  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
